# In this notebook I will attempt to run a prediciton on new mobile offers and chances are they will be completed

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import scipy
import matplotlib.pyplot as mlt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

# Define Color Pallete

In [2]:
color1 = '#00704A'
color2 = '#FF9FE5'
color3 = '#45062E'
backgroundColor = '#B8A085'
ccs = ['#ACDDA9', '#00704A', '#002F20']
dcs = ['#002619','#008256', '#00de92', '#a1ffdf', '#fdfffe']